# Data Preprocessing for Finetuning

## Overview

1. Process Raw Data: Parse original Oyez transcript jsonls and into processed jsonl. Each line has the following keys: `{"system_prompt","instruction", "output", "transcript_id", "justice"}`. **NOTE**: Optionally save to `finetuning_data_processed.jsonl`. Currently commented out this code to save on gitlfs storage.
1. Filter Processed Data: 
    1. Filter out non-question samples, i.e. where the response > 50 chars and has "?" in it and does not have "inaudible" in it.
    1. Filter out samples where the responding justice is not one of the current Supreme Court justices
1. Train/Validation/Test Split: Split in 80/10/10 ratio and produce `train.jsonl`, `val.jsonl`, and `test.jsonl`

**NOTE:** All output files are stored in `datasets/finetune/` and tracked with GIT LFS

##### Sample from `test.jsonl`
```
{
    "system_prompt":(
        "You are a Supreme Court Justice participating in oral arguments. "
        "Given a transcript excerpt and a Justice's name, generate the Justice's next question in response to the conversation history."
    ),
    "instruction":(
        "<context>\n"
        "<turn>Jameson R. Jones: Mr. Chief Justice, and may it please the Court: As some of this questioning indicated, if any party has standing under Section 43(a) of the Lanham Act, it's a party whose goods are misrepresented in false advertising. To remove any doubt about that question, Congress amended the statute in 1988 to ensure a cause of action when a false advertiser misrepresents the goods or commercial services of, quote, \u2036 another person \u2033, end quote. This Court's zone of interest analysis shows that parties whose goods are disparaged, either expressly or by necessary implication, must have standing to sue. Lexmark's simply wrong about the idea that the zone of interest analysis in the Lanham Act does not impose limits upon who may sue. As the hypothetical with respect to the Bailey's Ice Cream Parlor shows, you can look to the subject matter of the false advertisement to see whose goodwill and commercial activities are related to the falsity of the statement. And those who come within the falsity and the subject matter of the advertisement at issue should have standing, while those who may have tangential injuries would not.<\/turn>\n"
        "<turn>Justice Antonin Scalia: How do you -- how do you square that with the statutory provision that the purpose of the law is to prevent unfair competition? Unfair competition, not unfair trade practices? Unfair competition?<\/turn>\n"
        <turn>Jameson R. Jones: Where Section 45 says that it is designed to protect those engaged in such commerce from unfair competition, it's referring to what is defined in the operative text as unfair trade practices. Unfair competition involves specific measures, the use of falsities, that can injure parties who are not necessarily in competition with one another. The courts as a whole all agree that a competition requirement cannot be inferred into the false association cause of action that is also unfair competition that's part of Section 43(a). Section 43(a) goes to commercial activity. There is unfair competition in the sense that all of the activity under it is commercial and competitive in that sense. But some narrow form of competition between a plaintiff and a defendant for the purposes of standing is inconsistent with the structure of Section 43(a) and the text of the operative paragraph.<\/turn>\n"
        "<\/context>\n"
        "<justice>Justice Samuel A. Alito, Jr.<\/justice>\n"
        "Generate a question that Justice Samuel A. Alito, Jr. is likely to ask next."
    ),
    "output": (
        "Justice Samuel A. Alito, Jr.: Suppose the comments in this case only disparaged the cartridges themselves and not the chips. "
        "Then would the chip manufacturer, would your client have standing?"
    ),
    "transcript_id":"2013.12-873-t01",
    "justice":"Justice Samuel A. Alito, Jr."
}

```

In [1]:
import json
import os
import pandas as pd
from sklearn.model_selection import train_test_split

TRANSCRIPTS_DIR = "../transcripts_up_to_2024/"      # directory of raw JSONs of oral arguments
OUT_DIR = "../datasets/finetune"

def save_jsonl(df, filename):
    df.to_json(filename, orient="records", lines=True)

def read_jsonl(filename):
    with open(filename, "r") as f:
        data = [json.loads(line) for line in f]
    return data


## Step 1: Process Raw Data

In [ ]:
def get_formatted_text_of_turn(turn):
    '''
    Return all text within a turn as a dict denoting speaker, role and text.
    
    @param turn -- JSON representing a single speaker turn
    @return -- Dict with keys "speaker_name", "role", "text"
    '''
    if not turn["speaker"]: # Skip turns that have no speaker like "Laughter"
        return None
    
    if not turn["speaker"]["roles"]:
        role = "attorney"
    # check for Justice Amy Coney Barrett (formatted with the roles['2']) and otherwise justices with  roles[0]
    elif ('2' in turn["speaker"]["roles"] and turn["speaker"]["roles"]['2']["type"] == "scotus_justice") or turn["speaker"]["roles"][0]["type"] == "scotus_justice":
        role = "scotus_justice"
    
    if role == "scotus_justice":
        name = f"Justice {turn["speaker"]["name"]}"
    else:
        name = turn["speaker"]["name"]

    text = " ".join([block["text"] for block in turn["text_blocks"]])

    formatted_turn = {
        "speaker_name": name,
        "role": role,
        "text": text,
    }

    return formatted_turn

def format_conversation_segment(context_turns, justice_turn, transcript_id):
    '''
        Formats conversation context and the justice's response into fine-tuning format.
    '''

    justice_name = justice_turn["speaker_name"]
    
    formatted_data = {
        "system_prompt": (
            "You are a Supreme Court Justice participating in oral arguments. "
            "Given a transcript excerpt and a Justice's name, generate the Justice's next question in response to the conversation history."
        ),
        "instruction": (
            "<context>\n" +
            "\n".join([f"<turn>{turn['speaker_name']}: {turn['text']}</turn>" for turn in context_turns]) +
            "\n</context>\n" +
            f"<justice>{justice_name}</justice>\n" +
            f"Generate a question that {justice_name} is likely to ask next."
        ),
        "output": f"{justice_name}: {justice_turn['text']}",
        "transcript_id": transcript_id,
        "justice": justice_name,
    }
    
    return formatted_data

def process_turns(turn_data, transcript_id, max_context_chars=5000):
    '''
        Convert list of turns to expected format with a sliding window of max 3 turns
    '''
    formatted_data_list = []
    context_window = []
    context_char_count = 0

    for i in range(len(turn_data)):  
        current_turn = turn_data[i]

        # Only add this as sample if the current turn is spoken by a Justice and is not the first turn
        if current_turn["role"] == "scotus_justice" and len(context_window) > 0:
            formatted_data = format_conversation_segment(context_window, current_turn, transcript_id)
            formatted_data_list.append(formatted_data)

        # Add turn to context
        current_turn_text = f"{current_turn['speaker_name']}: {current_turn['text']}"
        context_window.append(current_turn)
        context_char_count += len(current_turn_text)

        # Ensure context stays within max_context_chars
        while context_char_count > max_context_chars and context_window:
            removed_turn = context_window.pop(0)
            removed_text = f"{removed_turn['speaker_name']}: {removed_turn['text']}"
            context_char_count -= len(removed_text)

    return formatted_data_list

def get_transcript_data(json_file_name):
    '''
    Parse JSON oral argument transcript into the formatted data needed for finetuning.

    @param json_file_name -- name of oral argument JSON file
    @return -- list of samples for finetuning
    '''

    transcript_file_path = TRANSCRIPTS_DIR + json_file_name
    with open(transcript_file_path) as json_file:
        transcript_json = json.load(json_file)
    
    transcript_id = json_file_name[:-5]
    formatted_data = []

    for section in [0, 1]:
        section_turns = transcript_json["transcript"]["sections"][section]["turns"]
        section_turns = [get_formatted_text_of_turn(turn) for turn in section_turns]
        section_turns = [turn for turn in section_turns if turn]
        formatted_data.extend(process_turns(section_turns, transcript_id))

    return formatted_data

'''
Parses then adds all historical transcript data into jsonl file with samples for finetuning
'''
data_transcripts = []
cases_dir = os.fsencode(TRANSCRIPTS_DIR)
success = fail = 0
for json_file_name in os.listdir(TRANSCRIPTS_DIR):
    if json_file_name.endswith('.json'):
        data_transcripts.extend(get_transcript_data(json_file_name))


# output_file = f"{OUT_DIR}/finetuning_data_processed.jsonl"
# with open(output_file, "w") as f:
#     for entry in data_transcripts:
#         f.write(json.dumps(entry) + "\n")

# print(f"Saved {len(data_transcripts)} fine-tuning examples to {output_file}")

Saved 230841 fine-tuning examples to ../datasets/finetune/finetuning_data_processed.jsonl


## Step 2: Filter Processed Data

In [ ]:
def filter_justices(sample):
    current_justices = {"Justice John G. Roberts, Jr.", "Justice Clarence Thomas", "Justice Samuel A. Alito, Jr.", "Justice Sonia Sotomayor", "Justice Elena Kagan", "Justice Neil Gorsuch", "Justice Brett M. Kavanaugh", "Justice Amy Coney Barrett", "Justice Ketanji Brown Jackson"}
    return sample in current_justices

def filter_questions(sample):
    ''' 
        Filter out data heuristically: >50 chars and has a '?' char to indicate a justice question.
    '''
    text = sample.split(': ', 1)[1]
    return len(text) > 50 and "?" in text and "inaudible" not in text.lower()

# # Load formatted dataset
# input_file = f"{OUT_DIR}/finetuning_data_processed.jsonl" 
# df = pd.read_json(input_file, lines=True) # Load data from saved finetuning_data_processed.jsonl file from previous step
df = pd.DataFrame(data_transcripts) # Load data directly from memory

print(f"# samples BEFORE filtering: {len(df)}")
# Filter data
df = df[df['output'].apply(filter_questions)]
df = df[df['justice'].apply(filter_justices)]
print(f"# samples AFTER filtering: {len(df)}")
print(f"# samples by {df['justice'].value_counts()}")

# Save filtered dataset
# output_file = f"{OUT_DIR}/finetuning_data_filtered.jsonl"
# save_jsonl(df, output_file)
# print(f"Saved filtered dataset to: {output_file}")

# samples BEFORE filtering: 230841
# samples AFTER filtering: 29680
# samples by justice
Justice John G. Roberts, Jr.     5997
Justice Sonia Sotomayor          5931
Justice Samuel A. Alito, Jr.     5746
Justice Elena Kagan              4122
Justice Neil Gorsuch             2432
Justice Brett M. Kavanaugh       1736
Justice Ketanji Brown Jackson    1358
Justice Amy Coney Barrett        1207
Justice Clarence Thomas          1151
Name: count, dtype: int64
Saved filtered dataset to: ../datasets/finetune/finetuning_data_filtered.jsonl


## Step 3: Train/Val/Test Split

In [ ]:
RANDOM_SEED = 42

# input_file = f"{OUT_DIR}/finetuning_data_filtered.jsonl"
# df = pd.read_json(input_file, lines=True) # Load data from saved finetuning_data_filtered.jsonl file from previous step

train_ratio = 0.80
val_ratio = 0.10
test_ratio = 0.10

# 0. get unique transcript_ids
transcript_ids = df['transcript_id'].unique()

# 1. split into train (80%) and eval (20%)
train_ids, eval_ids = train_test_split(
    transcript_ids, 
    test_size=(val_ratio + test_ratio), 
    random_state=RANDOM_SEED
)

# 2. split eval into validation (10%) and test (10%)
val_ids, test_ids = train_test_split(
    eval_ids, 
    test_size=(test_ratio / (val_ratio + test_ratio)), 
    random_state=RANDOM_SEED
)

train_data = df[df['transcript_id'].isin(train_ids)]
val_data = df[df['transcript_id'].isin(val_ids)]
test_data = df[df['transcript_id'].isin(test_ids)]

# 3. save splits
save_jsonl(train_data, f"{OUT_DIR}/train.jsonl")
save_jsonl(val_data, f"{OUT_DIR}/val.jsonl")
save_jsonl(test_data, f"{OUT_DIR}/test.jsonl")

print(f"Dataset split complete:\nTrain: {len(train_data)}\nValidation: {len(val_data)}\nTest: {len(test_data)}")
print(f"\nTRANSCRIPT ID counts:\nTotal: {len(transcript_ids)}\nTrain: {len(train_ids)}\nValidation: {len(val_ids)}\nTest: {len(test_ids)}")

Dataset split complete:
Train: 23365
Validation: 3402
Test: 2913

TRANSCRIPT ID counts:
Total: 1294
Train: 1035
Validation: 129
Test: 130


## Step 4: Create small test subsample

In [10]:
RANDOM_SEED = 42
SAMPLE_SIZE = 100

input_file = f"{OUT_DIR}/test.jsonl"
df = pd.read_json(input_file, lines=True)

test_sample = df.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED)
save_jsonl(test_sample, f"{OUT_DIR}/test_{SAMPLE_SIZE}.jsonl")

In [11]:
# test_sample.info()

## Debug

### Verify proportions of justice and years across splits

In [ ]:
def check_proportions(train_data, val_data, test_data, column):
    train_proportions = train_data[column].value_counts(normalize=True).sort_index()
    val_proportions = val_data[column].value_counts(normalize=True).sort_index()
    test_proportions = test_data[column].value_counts(normalize=True).sort_index()

    proportion_df = pd.DataFrame({
        'Train': train_proportions,
        'Validation': val_proportions,
        'Test': test_proportions
    })
    proportion_df = proportion_df * 100
    print(proportion_df)
    proportion_df.plot(kind='bar', figsize=(8, 4))

JUSTICE PROPORTIONS:

In [7]:
# check_proportions(train_data, val_data, test_data, column='justice')

YEAR PROPORTIONS:

In [6]:
# def add_year_column(dataframe):
#     dataframe['year'] = dataframe['transcript_id'].apply(lambda x: x.split('.')[0])
#     return dataframe

# train_data = add_year_column(train_data)
# val_data = add_year_column(val_data)
# test_data = add_year_column(test_data)

# check_proportions(train_data, val_data, test_data, column='year')

### Debug Raw data processing

In [81]:
# # debugging data
# turns_data = {
#     "turns": [
#         {
#             "speaker_name": "MR. McCONNELL",
#             "role": "counsel",
#             "text": "1. Mr. Chief Justice, and may it please the Court to understand..."
#         },
#         {
#             "speaker_name": "JUSTICE KENNEDY",
#             "role": "scotus_justice",
#             "text": "2. But we do have the problem of the stipulation..."
#         },
#         {
#             "speaker_name": "MR. McCONNELL",
#             "role": "counsel",
#             "text": "3. Happy to, Justice Kennedy. If you just look with me at Joint Stipulation 17..."
#         },
#         {
#             "speaker_name": "JUSTICE SCALIA",
#             "role": "scotus_justice",
#             "text": "4. But isn't the all-comers policy broader than the nondiscrimination policy?"
#         },
#         {
#             "speaker_name": "JUSTICE KENNEDY",
#             "role": "scotus_justice",
#             "text": "5. But we do have the problem of the stipulation..."
#         },
#         {
#             "speaker_name": "JUSTICE SCALIA",
#             "role": "scotus_justice",
#             "text": "6. But isn't the all-comers policy broader than the nondiscrimination policy?"
#         },
#         {
#             "speaker_name": "MR. McCONNELL",
#             "role": "counsel",
#             "text": "7. TAKE 3 Happy to, Justice Kennedy. If you just look with me at Joint Stipulation 17..."
#         },
#         {
#             "speaker_name": "JUSTICE SCALIA",
#             "role": "scotus_justice",
#             "text": "8. ROUND @ But isn't the all-comers policy broader than the nondiscrimination policy?"
#         },
#         {
#             "speaker_name": "MR. McCONNELL",
#             "role": "counsel",
#             "text": "9. TAKE 4 Happy to, Justice Kennedy. If you just look with me at Joint Stipulation 17..."
#         },
#         {
#             "speaker_name": "JUSTICE KENNEDY",
#             "role": "scotus_justice",
#             "text": "10. But we do have the problem of the stipulation..."
#         },
#         {
#             "speaker_name": "JUSTICE SCALIA",
#             "role": "scotus_justice",
#             "text": "11. But isn't the all-comers policy broader than the nondiscrimination policy?"
#         },
#         {
#             "speaker_name": "MR. McCONNELL",
#             "role": "counsel",
#             "text": "12. Happy to, Justice Kennedy. If you just look with me at Joint Stipulation 17..."
#         },
#     ]
# }

# MAX_CONTEXT_CHARS = 500

# formatted_data_list = []
# context_window = []
# context_char_count = 0

# for i in range(len(turns_data)):  
#     current_turn = turns_data[i]

#     if current_turn["role"] == "scotus_justice":
#         formatted_data = format_conversation_segment(context_window, current_turn, 'debug_transcript_id')
#         formatted_data_list.append(formatted_data)

#     current_turn_text = f"{current_turn['speaker_name']}: {current_turn['text']}"
#     context_window.append(current_turn)
#     context_char_count += len(current_turn_text)

#     while context_char_count > MAX_CONTEXT_CHARS and context_window:
#         removed_turn = context_window.pop(0)
#         removed_text = f"{removed_turn['speaker_name']}: {removed_turn['text']}"
#         context_char_count -= len(removed_text)


In [ ]:
# for i in range(len(formatted_data_list)):
#     print(formatted_data_list[i]['messages'][1]['content'])
#     print(formatted_data_list[i]['messages'][2]['content'])
#     print("\n\n")